### Pydantic Models

Models are classes which inherit from the BaseModel class

In [1]:
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    id: int
    name = 'Suvro Banerjee'

In [3]:
user = User(id = '123')
user_x = User(id = '123.45')

ValidationError: 1 validation error for User
id
  value is not a valid integer (type=type_error.integer)

In [5]:
user_x = User(id = 123.45)

In [6]:
assert user.id == 123

In [8]:
assert user_x.id == 123  # note the float is casted to int

In [9]:
assert isinstance(user_x.id, int)

In [10]:
user.name

'Suvro Banerjee'

In [11]:
user.dict()

{'id': 123, 'name': 'Suvro Banerjee'}

In [13]:
assert user.dict() == dict(user) == {'id': 123, 'name': 'Suvro Banerjee'}

In [14]:
user.json()

'{"id": 123, "name": "Suvro Banerjee"}'

### Recursive Models

For complex hierarchical data strcutures

In [15]:
from typing import Optional
from pydantic import BaseModel

In [16]:
class Foo(BaseModel):
    count: int
    size: Optional[float] = None
    
class Bar(BaseModel):
    apple = 'x'
    banana = 'y'
    
class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]

In [17]:
m = Spam(foo = {'count': 4},
        bars = [{'apple': 'x1'}, {'apple': 'x2'}])
print(m)

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]


In [18]:
print(m.json())

{"foo": {"count": 4, "size": null}, "bars": [{"apple": "x1", "banana": "y"}, {"apple": "x2", "banana": "y"}]}


In [19]:
print(m.dict())

{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


### ORM Mode (aka Arbitrary Class Instances)
ORM stands for Object-Relational Mapping. It is a programming technique that allows developers to interact with databases using an object-oriented paradigm, rather than relying on traditional SQL statements. An ORM tool provides a mapping between the database schema and the programming language's object model, so that developers can create, read, update, and delete records in the database using objects and methods in their programming language of choice.

In an ORM, each table in the database is represented by a class in the programming language, and each row in the table is represented by an instance of that class. ORM tools typically provide a range of methods to interact with the database, such as create, read, update, and delete methods that allow developers to manipulate the database records using object-oriented syntax.

ORMs provide several benefits over traditional SQL-based database interactions, including:

Reduced code complexity: ORMs abstract away the complexity of SQL queries and provide a more intuitive and concise programming interface.

Portability: ORMs can be used with multiple database management systems (DBMS) without requiring significant changes to the code.

Maintenance: ORMs provide a layer of abstraction that separates the database schema from the application logic, making it easier to maintain and update the application as the database schema changes.

Some popular ORM tools include Hibernate (for Java), Entity Framework (for .NET), Django ORM (for Python), and ActiveRecord (for Ruby on Rails).


The below example uses SQLAlchemy as an ORM-

**SQLAlchemy** is a popular open-source Object-Relational Mapping (ORM) library for Python. It provides a set of tools and utilities for interacting with relational databases using a high-level, object-oriented interface. SQLAlchemy supports a wide range of database systems, including PostgreSQL, MySQL, Oracle, Microsoft SQL Server, and SQLite.

One of the key features of SQLAlchemy is its ability to generate SQL queries dynamically based on Python code. This allows developers to write complex database queries in Python, rather than directly in SQL, and to easily change the queries without having to manually modify the SQL code. SQLAlchemy also provides support for advanced SQL features such as transactions, joins, and subqueries.

In addition to its ORM capabilities, SQLAlchemy also provides a range of other tools and utilities for working with databases, including:

A SQL expression language that allows developers to generate SQL statements programmatically.

A schema abstraction layer that simplifies the process of creating, modifying, and querying database schemas.

Connection pooling and caching to improve performance and reduce database load.

Integration with popular web frameworks such as Flask and Django.

Overall, SQLAlchemy is a powerful and flexible library that makes it easier to work with relational databases in Python, whether you are building a small web application or a large-scale data-driven system.


In [20]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

In [21]:
Base = declarative_base()

/var/folders/xc/x2qm7j8j4k75m7htc8v3w1vc0000gn/T/ipykernel_26515/4196137762.py:1: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [22]:
class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key = True, nullable = False)
    public_key = Column(String(20), index = True, nullable = False, unique = True)
    name = Column(String(63), unique = True)
    domains = Column(ARRAY(String(255)))

In [23]:
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length = 20)
    name: constr(max_length = 63)
    domains: list[constr(max_length = 255)]
    
    class Config:
        orm_mode = True

In Pydantic, Constr is a class that represents a field that has a set of constraints. It is used to define validation rules for data that is being parsed or serialized.

In [24]:
co_orm = CompanyOrm(
        id = 123,
        public_key = 'foobar',
        name = 'Testing',
        domains = ['example.com', 'foobar.com'],
)

In [25]:
print(co_orm)

In [26]:
co_model = CompanyModel.from_orm(co_orm)
print(co_model)

id=123 public_key='foobar' name='Testing' domains=['example.com', 'foobar.com']


Let's take another example -